### Load Dataset Files

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
project_path = "/content/drive/My Drive/AIML_Labs/Dog_breed/"

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

In [0]:
import pandas as pd

In [0]:
labels = pd.read_csv('./labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [0]:
breed_count = lables['breed'].value_counts()

breed_count.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

In [0]:
breed_count.shape

(120,)

### One-hot encodings of labels

In [0]:
import numpy as np
targets = pd.Series(lables['breed'])
one_hot = pd.get_dummies(targets)
one_hot_labels = np.asarray(one_hot)

In [0]:
one_hot_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

Or you can use below code also for creating one hot encodings

In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['breed'])
print(integer_encoded)

[19 37 85 ...  3 75 28]


In [0]:
import keras
y = keras.utils.to_categorical(integer_encoded, num_classes=None, dtype='int')

In [0]:
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
img_rows=128
img_cols=128
num_channel=1

In [0]:
from tqdm import tqdm
import cv2
x_feature = []
y_feature = []

i = 0 # initialisation
for f, img in tqdm(lables.values): # f for format ,jpg
    train_img = cv2.imread('./train/{}.jpg'.format(f),0)
    label = one_hot_labels[i]
    train_img_resize = cv2.resize(train_img, (img_rows, img_cols)) 
    x_feature.append(train_img_resize)
    y_feature.append(label)
    i += 1

100%|██████████| 10222/10222 [00:15<00:00, 662.47it/s]


In [0]:
x_train_data = np.array(x_feature, np.float32) / 255.   # /= 255 for normolisation
print (x_train_data.shape)
x_train_data = np.expand_dims(x_train_data, axis = 3) # for keras to given input to Conv2D layer
print (x_train_data.shape)


(10222, 128, 128)
(10222, 128, 128, 1)


In [0]:
y_train_data = np.array(y_feature)

y_train_data.shape

(10222, 120)

### Split the training and validation data here

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_data, y_train_data, test_size=0.2, random_state=2)
print (x_train.shape)
print (x_val.shape)

(8177, 128, 128, 1)
(2045, 128, 128, 1)


### Test features

In [0]:
submission = pd.read_csv('sample_submission.csv')
test_img = submission['id']
submission.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [0]:
x_test_feature = []

i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:15<00:00, 664.24it/s]


In [0]:
x_test_data = np.array(x_test_feature, np.float32) / 255. 
print (x_test_data.shape)
x_test_data = np.expand_dims(x_test_data, axis = 3)
print (x_test_data.shape)

(10357, 128, 128)
(10357, 128, 128, 1)


In [0]:
from keras.models import Sequential  # initial NN
from keras.layers import Dense, Dropout # construct each layer
from keras.layers import Conv2D # swipe across the image by 1
from keras.layers import MaxPooling2D # swipe across by pool size
from keras.layers import Flatten, GlobalAveragePooling2D

In [0]:
model = Sequential()

model.add(Conv2D(32, (5,5), activation ='relu', input_shape = (img_rows, img_cols, num_channel))) 
model.add(MaxPooling2D(pool_size=3))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.8))

model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.8))

model.add(Flatten()) 
# fully connected layer 
model.add(Dense(units = 500, activation = 'relu')) 
model.add(Dropout(0.8))
# output layer
model.add(Dense(units = 120, activation = 'softmax')) 

In [0]:
from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"]) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_119 (Conv2D)          (None, 124, 124, 32)      832       
_________________________________________________________________
max_pooling2d_98 (MaxPooling (None, 41, 41, 32)        0         
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 41, 41, 64)        32832     
_________________________________________________________________
max_pooling2d_99 (MaxPooling (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_121 (Conv2D)          (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_100 (MaxPoolin (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_60 (Dropout)         (None, 10, 10, 128)       0         
__________

In [0]:
batch_size = 128
nb_epochs = 20
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epochs, 
                    validation_data=(x_val, y_val),
                    initial_epoch=0)

Train on 8177 samples, validate on 2045 samples
Epoch 1/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.8173 - acc: 0.0094 - val_loss: 4.7888 - val_acc: 0.0088
Epoch 2/20
8177/8177 [==============================] - 7s 852us/step - loss: 4.7811 - acc: 0.0117 - val_loss: 4.7891 - val_acc: 0.0117
Epoch 3/20
8177/8177 [==============================] - 7s 852us/step - loss: 4.7795 - acc: 0.0097 - val_loss: 4.7902 - val_acc: 0.0088
Epoch 4/20
8177/8177 [==============================] - 7s 853us/step - loss: 4.7776 - acc: 0.0116 - val_loss: 4.7915 - val_acc: 0.0117
Epoch 5/20
8177/8177 [==============================] - 7s 850us/step - loss: 4.7767 - acc: 0.0117 - val_loss: 4.7926 - val_acc: 0.0117
Epoch 6/20
8177/8177 [==============================] - 7s 848us/step - loss: 4.7771 - acc: 0.0115 - val_loss: 4.7928 - val_acc: 0.0117
Epoch 7/20
8177/8177 [==============================] - 7s 853us/step - loss: 4.7771 - acc: 0.0114 - val_loss: 4.7926 - val_acc: 0.0117
Ep

### Using Data Augmentation

In [0]:
from keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from keras.layers import ZeroPadding2D

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen=ImageDataGenerator(rescale=1./255)

In [0]:
training_set=train_datagen.flow(x_train,y=y_train,batch_size=32)
testing_set=test_datagen.flow(x_val,y=y_val,batch_size=32)

In [0]:
model=Sequential()

# model.add(ZeroPadding2D((1,1),input_shape=(128,128,1)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(ZeroPadding2D(padding=(1,1)))

model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(120,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_15 (ZeroPaddi (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_146 (Conv2D)          (None, 128, 128, 32)      320       
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 130, 130, 32)      0         
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 128, 128, 32)      9248      
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_115 (MaxPoolin (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 61, 61, 64)        18496     
__________

In [0]:
history=model.fit_generator(training_set,
                      steps_per_epoch = 500,
                      validation_data = testing_set,
                      validation_steps = 4,
                      epochs = 20,
                      verbose = 1)

### Using Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input


# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

In [0]:
from tqdm import tqdm
import cv2
x_feature = []
y_feature = []

i = 0 # initialisation
for f, img in tqdm(lables.values): # f for format ,jpg
    train_img = cv2.imread('./train/{}.jpg'.format(f),1)
    label = one_hot_labels[i]
    train_img_resize = cv2.resize(train_img, (img_rows, img_cols)) 
    x_feature.append(train_img_resize)
    y_feature.append(label)
    i += 1

100%|██████████| 10222/10222 [00:27<00:00, 365.27it/s]


### Get Image embedding from the pre-trained model and use that as input to present model

In [0]:
from keras.preprocessing import image

X_arr = []
# X_arr = base_model.predict(preprocess_input(img_to_array(image.load_img('./train/'+labels['id'][0]+'.jpg', target_size=[128,128]))))

for id, label in tqdm(labels.values):
    X_arr.append(base_model.predict(preprocess_input(np.expand_dims(img_to_array(image.load_img('./train/'+id+'.jpg', target_size=[128,128])), axis=0)))[0])
X = pd.DataFrame(X_arr)

100%|██████████| 10222/10222 [02:44<00:00, 62.23it/s]


In [0]:
from keras.layers import Activation
model = Sequential([
    Dense(1024, input_shape=(512,)),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(120),
    Activation('softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
X_train = X
y_train = y_feature

In [0]:
model.fit(X_train, np.asarray(y_train), epochs=100, batch_size=100, verbose=True)

Epoch 1/100
10222/10222 [==============================] - 3s 334us/step - loss: 4.6754 - acc: 0.1445
Epoch 2/100
10222/10222 [==============================] - 1s 88us/step - loss: 2.2579 - acc: 0.4205
Epoch 3/100
10222/10222 [==============================] - 1s 88us/step - loss: 1.5547 - acc: 0.5739
Epoch 4/100
10222/10222 [==============================] - 1s 88us/step - loss: 1.1363 - acc: 0.6709
Epoch 5/100
10222/10222 [==============================] - 1s 85us/step - loss: 0.7701 - acc: 0.7755
Epoch 6/100
10222/10222 [==============================] - 1s 86us/step - loss: 0.4760 - acc: 0.8640
Epoch 7/100
10222/10222 [==============================] - 1s 89us/step - loss: 0.2763 - acc: 0.9288
Epoch 8/100
10222/10222 [==============================] - 1s 86us/step - loss: 0.1816 - acc: 0.9547
Epoch 9/100
10222/10222 [==============================] - 1s 88us/step - loss: 0.1325 - acc: 0.9685
Epoch 10/100
10222/10222 [==============================] - 1s 87us/step - loss: 0.0951 - 